In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, shutil, time, pdb, random
import scipy.stats as stats 
import scipy

from math import pi
from datetime import datetime
from collections import OrderedDict
import pickle

import torch
from torch.utils.data import TensorDataset, DataLoader 

from plot_utils import table_of_predictions_for_metric

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from seasonal_analysis import * 

from sklearn.metrics import mean_squared_error as mse

import matplotlib
matplotlib.rc_file_defaults()
%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns
from tabulate import tabulate

pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

# Detect device.
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

### Import data

In [2]:
# Parameters
start="1998-01-01"
end="2013-12-31"

# TRAIN_PATH = "../data/pickle/df_stations_all_nonzero_extended.pkl"
# TEST_PATH = "../data/pickle/df_stations_val_all_nonzero_extended.pkl"

TRAIN_PATH = '../data/pickle/observations_with_WRF_norris.pkl'

In [3]:
# st = create_station_dataframe(TRAIN_PATH, start, end, add_yesterday=True, basin_filter = None, filter_incomplete_years = True)

data = DataPreprocessing(train_path=TRAIN_PATH, start=start, end=end, 
                add_yesterday = True, 
                basin_filter = None, 
                split_bias_corrected_only = True, 
                filter_incomplete_years = True, 
                include_non_bc_stations = True, 
                split_by = 'station')

In [92]:
data.split_stations()

35 stations used for training, 6 used for validation, and 6 used testing


### Split data into held out sets for K-fold cross validation

### Prepare data

In [93]:
predictors = [
                'wrf_prcp',
                'wrf_prcp_-2','wrf_prcp_-1','wrf_prcp_1','wrf_prcp_2',
                'Z',
                #'doy',
                'doy_sin',
                'doy_cos',
                'X',
                'Y',
                'aspect',
                'slope',
                'year',
                'era5_u','era5_u_-1','era5_u_-2','era5_u_1','era5_u_2',
                'era5_v','era5_v_-1','era5_v_-2','era5_v_1','era5_v_2',
             ]

predictors = [ 
                'doy_sin',
                'doy_cos',
                'Z',
                'X',
                'Y',
                'aspect',
                'slope',
                'year',
                'CWV_norris', 
                'RH2_norris', 'RH500_norris', 
                'T2_norris', 'T2max_norris', 'T2min_norris', 'Td2_norris', 
                'precip_norris', 'rain_norris', 
                'u10_norris', 'u500_norris', 'v10_norris', 'v500_norris',
              ]

# predictors.append('obs_yesterday')

predictand = ['Prec']

data.input_data(predictors, predictand, sort_by_quantile=False)

## Multi-Run: Train model with different hyperparameters

### Model run

In [117]:
params = OrderedDict(
    lr = [0.005]
    ,batch_size = [16, 128]
    ,likelihood_fn = ['bgmm'] #, 'bernoulli_loggaussian', 'b2gmm'] #['bernoulli_loggaussian']
    ,hidden_channels = [[50]] #[[10],[30],[50],[100],[10,10],[30,30],[50,50],[100,100]]
    ,dropout_rate = [0, 0.50]
    ,linear_model = [False] #['True','False']
    #,k = [0]
    ,k = list(range(10))
)

epochs = 10

In [118]:
st_test, predictions = multirun(data, predictors, params, epochs, split_by='station',
                                sequential_samples=False)

,run,epoch,loss,valid_loss,epoch duration,run duration,lr,batch_size,likelihood_fn,hidden_channels,dropout_rate,linear_model,k
0,1,1,1.207078,1.015417,6.045622,6.067245,0.005,16,bgmm,[50],0.0,False,0
1,1,2,1.152805,1.026391,6.793367,15.965029,0.005,16,bgmm,[50],0.0,False,0
2,1,3,1.148003,1.023731,5.310157,22.339536,0.005,16,bgmm,[50],0.0,False,0
3,1,4,1.145700,1.003636,7.458650,29.851993,0.005,16,bgmm,[50],0.0,False,0
4,1,5,1.143256,1.009415,6.891253,38.835174,0.005,16,bgmm,[50],0.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,40,6,1.185218,1.020388,1.098879,16.835052,0.005,128,bgmm,[50],0.5,False,9
396,40,7,1.184993,1.007461,1.096215,19.004863,0.005,128,bgmm,[50],0.5,False,9
397,40,8,1.184317,1.025990,1.052561,23.148640,0.005,128,bgmm,[50],0.5,False,9
398,40,9,1.181625,1.030684,1.070316,25.291245,0.005,128,bgmm,[50],0.5,False,9


In [119]:
for run in predictions.keys():
    for i in range(len(params['k'])):
        predictions[run][f'k{i}']['k_fold'] = i
        if i == 0:
            predictions[run]['k_all'] = predictions[run][f'k{i}']
        else:
            predictions[run]['k_all'] = predictions[run]['k_all'].append(predictions[run][f'k{i}'])

In [120]:
# Table of predictions
seasons = ['Winter (JFM)', 'Premonsoon (AM)', 'Monsoon (JJAS)','Postmonsoon (OND)']
# table_of_predictions(predictions, seasons, sample_cols=['sample'])

In [121]:
# for key in predictions.keys():
#     predictions[key]['k_all'].rename(columns={"sample": "sample_0"}, inplace=True)

In [122]:
# predictions['bgmm_[50]_False']['k_all'].rename(columns={"wrf_prcp": "bannister_wrf_prcp", "precip_norris": "wrf_prcp"}, inplace=True)

In [123]:
n_samples = 10
sample_cols = [f'sample_{i}' for i in range(n_samples)]
add_cols = []

columns = ['Prec','wrf_prcp','wrf_bc_prcp','precip_norris'] #+ sample_cols + add_cols

In [124]:
table_of_predictions_ks_test(predictions, seasons, columns, sample_cols, add_cols)

bgmm_[50]_NL_B=16_D=0
bgmm_[50]_NL_B=16_D=0.5
bgmm_[50]_NL_B=128_D=0
bgmm_[50]_NL_B=128_D=0.5
Model                     Winter (JFM) mean    Winter (JFM) median    Premonsoon (AM) mean    Premonsoon (AM) median    Monsoon (JJAS) mean    Monsoon (JJAS) median    Postmonsoon (OND) mean    Postmonsoon (OND) median
------------------------  -------------------  ---------------------  ----------------------  ------------------------  ---------------------  -----------------------  ------------------------  --------------------------
Bann                      0.3665               0.0814                 0.2890                  0.1557                    0.1368                 0.0929                   0.4909                    0.1071
BannCorr                  0.3762               0.0814                 0.3975                  0.1148                    0.1833                 0.0898                   0.4896                    0.1071
Norr                      0.1955               0.0998           

In [125]:
table_of_predictions_for_metric(predictions, seasons, columns, n_samples, sample_cols, add_cols, metric = 'smape', prefix='smape')

Model                     Winter (JFM) mean    Winter (JFM) median    Premonsoon (AM) mean    Premonsoon (AM) median    Monsoon (JJAS) mean    Monsoon (JJAS) median    Postmonsoon (OND) mean    Postmonsoon (OND) median
------------------------  -------------------  ---------------------  ----------------------  ------------------------  ---------------------  -----------------------  ------------------------  --------------------------
Bann                      0.25                 0.21                   0.42                    0.39                      0.22                   0.20                     0.39                      0.35
BannCorr                  0.21                 0.15                   0.31                    0.28                      0.16                   0.15                     0.35                      0.28
Norr                      0.22                 0.18                   0.42                    0.41                      0.22                   0.19               

In [126]:
table_of_predictions_for_metric(predictions, seasons, columns, n_samples, sample_cols, add_cols, metric = 'edd', prefix='edd')

Model                     Winter (JFM) mean    Winter (JFM) median    Premonsoon (AM) mean    Premonsoon (AM) median    Monsoon (JJAS) mean    Monsoon (JJAS) median    Postmonsoon (OND) mean    Postmonsoon (OND) median
------------------------  -------------------  ---------------------  ----------------------  ------------------------  ---------------------  -----------------------  ------------------------  --------------------------
Bann                      5.72                 4.00                   3.81                    3.00                      13.41                  11.00                    2.14                      2.00
BannCorr                  5.72                 4.00                   3.81                    3.00                      13.41                  11.00                    2.14                      2.00
Norr                      11.08                10.00                  9.62                    9.00                      34.36                  33.50              

In [160]:
a = pd.read_csv('results.csv').groupby(['k','run']).agg({'valid_loss': 'min', 
                                               'hidden_channels': 'first', 
                                               'likelihood_fn': 'first',
                                               'lr':'first',
                                               'batch_size':'first'})

a.groupby(['run']).agg({'valid_loss': 'mean', 
                   'hidden_channels': 'first', 
                   'likelihood_fn': 'first',
                   'lr':'first',
                   'batch_size':'first'}
                   ).sort_values('valid_loss').reset_index()

,run,valid_loss,hidden_channels,likelihood_fn,lr,batch_size
0,29,0.987930,[50],bgmm,0.005,128
1,21,0.988559,[50],bgmm,0.005,128
2,39,0.997434,[50],bgmm,0.005,128
3,9,1.000607,[50],bgmm,0.005,16
4,31,1.001022,[50],bgmm,0.005,128
5,27,1.001739,[50],bgmm,0.005,128
6,1,1.003636,[50],bgmm,0.005,16
7,28,1.005014,[50],bgmm,0.005,128
8,40,1.007461,[50],bgmm,0.005,128
9,19,1.009724,[50],bgmm,0.005,16
